# MNIST - MLP (multilayer perceptron)
This notebook introduces the MNIST handwritten digits dataset and multilayer perceptron classifier. 

In [34]:
model_name = "bge-large-en-v1.5"


In [35]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
%matplotlib inline
import torch
import os
import matplotlib.pyplot as plt

root_notebook_folder = os.path.abspath('.')
data_folder = os.path.abspath('./local_data')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Load Enbeddings

In [37]:
model_name_parquet = 'embeded_final_' + model_name + '.parquet'

In [38]:
# file_name = os.path.join(data_folder, 'embeded_final.csv')

file_name = os.path.join(data_folder, 'data_for_supervised', model_name_parquet)

import pandas as pd
data = pd.read_parquet(file_name)


In [39]:
data

,0,1,2,3,4,5,6,7,8,9,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,movement_label
0,-0.012828,0.002089,-0.023391,0.023389,-0.020759,0.014113,-0.004620,-0.008366,0.017626,0.049671,...,-0.036811,-0.030837,-0.009902,0.030089,0.047236,0.023684,-0.022334,-0.022012,-0.011921,NaN
1,-0.013832,0.001074,-0.024396,0.020885,-0.020149,0.012370,-0.003974,-0.007666,0.019587,0.048920,...,-0.037093,-0.029989,-0.009730,0.032658,0.046412,0.024297,-0.021598,-0.022996,-0.013874,1.0
2,-0.011602,0.003142,-0.021433,0.023721,-0.020300,0.016250,-0.002687,-0.008697,0.019624,0.049712,...,-0.035008,-0.027318,-0.010153,0.032859,0.044599,0.022673,-0.020713,-0.022317,-0.011632,NaN
3,-0.013550,-0.001042,-0.025677,0.022362,-0.020639,0.014104,-0.005741,-0.011706,0.018337,0.050350,...,-0.035246,-0.031524,-0.009279,0.029467,0.048446,0.022119,-0.020918,-0.020293,-0.011769,NaN
4,-0.013175,-0.000562,-0.025176,0.025241,-0.020414,0.013714,-0.006912,-0.009624,0.017347,0.048854,...,-0.035813,-0.031052,-0.008171,0.031470,0.047521,0.022224,-0.019980,-0.022457,-0.012740,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8159,-0.016140,0.002765,-0.021068,0.014904,-0.024773,0.012019,0.000108,0.001141,0.008714,0.052986,...,-0.022750,-0.025471,0.001547,0.032937,0.043843,0.016535,-0.016916,-0.019180,-0.002369,NaN
8160,-0.015037,0.001795,-0.025471,0.017483,-0.023001,0.015526,0.001447,0.003537,0.011914,0.048330,...,-0.025056,-0.028140,0.002017,0.033176,0.044044,0.013970,-0.020287,-0.020002,-0.002353,NaN
8161,-0.013757,0.001451,-0.023787,0.018711,-0.026334,0.016010,0.001308,0.002894,0.010190,0.048968,...,-0.027970,-0.026673,0.001311,0.034371,0.045282,0.016877,-0.021554,-0.021788,-0.000572,NaN
8162,-0.018161,-0.001021,-0.023106,0.022046,-0.022326,0.014735,0.001045,0.000673,0.009909,0.050792,...,-0.029186,-0.027653,-0.000698,0.034111,0.043050,0.015890,-0.018968,-0.018492,-0.004089,0.0


In [40]:
data['movement_label'] = data['movement_label'].fillna(2)

In [41]:
data['movement_label']

0       2.0
1       1.0
2       2.0
3       2.0
4       1.0
       ... 
8159    2.0
8160    2.0
8161    2.0
8162    0.0
8163    1.0
Name: movement_label, Length: 8164, dtype: float64

In [42]:
import numpy as np
len_x = len(data.columns) - 1
X = np.vstack(data.values[:,0:len_x] ).astype(np.float32)
y = data["movement_label"].astype(int).values

X.shape[1]

1024

In [43]:
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=42, stratify=y_temp
)

In [45]:
# pip install torch scikit-learn pandas numpy
import ast
import os
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# ====== 1) קלט: השתמשו ב-df קיים, או טענו מקובץ ======
# אם כבר יש לכם df בזיכרון, השאירו שורה זו כמובנת מאליה.
# אחרת, בטלו הערה לשורה הבאה והכניסו את הנתיב:
# df = pd.read_csv("your_embeddings.csv")

# שמות עמודות (התאימו אם אצלכם שונים)
EMB_COL = "embedding"
LABEL_COL = "movement_label"

# אם ה-embedding נשמר כמחרוזת, נהפוך לרשימה



# המרת ל-numpy
len_x = len(data.columns) - 1
X = np.vstack(data.values[:,0:len_x] ).astype(np.float32)
y = data["movement_label"].astype(int).values

# (אופציונלי) אם יש NaN בלייבל ואתם רוצים להחליף ל-2:
# y = np.where(pd.isna(y), 2, y).astype(int)

num_classes = len(np.unique(y))
print("Shape X:", X.shape, "classes:", np.unique(y))

# ====== 2) פיצול ל-train/val/test בשכבות ======
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=42, stratify=y_temp
)

# ====== 3) נרמול/סטנדרטיזציה (למרות ש-embeddings לרוב מנורמלים) ======
scaler = StandardScaler(with_mean=True, with_std=True)
X_train = scaler.fit_transform(X_train)
X_val   = scaler.transform(X_val)
X_test  = scaler.transform(X_test)

# ====== 4) PyTorch Dataset/DataLoader ======
class NPDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X).float()
        self.y = torch.from_numpy(y).long()
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_ds = NPDataset(X_train, y_train)
val_ds   = NPDataset(X_val, y_val)
test_ds  = NPDataset(X_test, y_test)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=128, shuffle=False)
test_loader  = DataLoader(test_ds, batch_size=128, shuffle=False)

# ====== 5) מודל MLP פשוט ======
input_dim = X.shape[1]

class MLP(nn.Module):
    def __init__(self, in_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, num_classes)
        )
    def forward(self, x):
        return self.net(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLP(input_dim, num_classes).to(device)

# ====== 6) איזון מחלקות (class weights) ======
class_counts = np.bincount(y_train, minlength=num_classes)
class_weights = (len(y_train) / (num_classes * np.clip(class_counts, 1, None))).astype(np.float32)
class_weights_t = torch.tensor(class_weights, dtype=torch.float32, device=device)
criterion = nn.CrossEntropyLoss(weight=class_weights_t)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

# ====== 7) לולאת אימון עם Early Stopping ======
best_val_loss = float("inf")
patience = 50
patience_counter = 0
EPOCHS = 20

def eval_loader(model, loader):
    model.eval()
    all_logits, all_y = [], []
    val_loss = 0.0
    with torch.no_grad():
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            logits = model(xb)
            loss = criterion(logits, yb)
            val_loss += loss.item() * yb.size(0)
            all_logits.append(logits.cpu())
            all_y.append(yb.cpu())
    val_loss /= len(loader.dataset)
    y_true = torch.cat(all_y).numpy()
    y_pred = torch.argmax(torch.cat(all_logits), dim=1).numpy()
    acc = accuracy_score(y_true, y_pred)
    f1m = f1_score(y_true, y_pred, average="macro")
    return val_loss, acc, f1m

for epoch in range(1, EPOCHS+1):
    model.train()
    train_loss = 0.0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        logits = model(xb)
        loss = criterion(logits, yb)
        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=2.0)
        optimizer.step()
        train_loss += loss.item() * yb.size(0)
    train_loss /= len(train_loader.dataset)
    print(f"Epoch {epoch:03d} | loss={loss} | ")

    val_loss, val_acc, val_f1 = eval_loader(model, val_loader)
    # print(f"Epoch {epoch:03d} | train_loss={train_loss:.4f} | val_loss={val_loss:.4f} | val_acc={val_acc:.4f} | val_f1={val_f1:.4f}")

    # early stopping
    if val_loss < best_val_loss - 1e-4:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(model.state_dict(), "best_mlp.pth")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping.")
            break

# ====== 8) טעינת המודל הטוב ובדיקה על ה-Test ======
model.load_state_dict(torch.load("best_mlp.pth", map_location=device))

test_loss, test_acc, test_f1 = eval_loader(model, test_loader)
print(f"\nTEST | loss={test_loss:.4f} | acc={test_acc:.4f} | f1_macro={test_f1:.4f}")

# Confusion Matrix + דו"ח
model.eval()
y_true_all, y_pred_all = [], []
with torch.no_grad():
    for xb, yb in test_loader:
        xb = xb.to(device)
        logits = model(xb).cpu()
        preds = torch.argmax(logits, dim=1)
        y_true_all.append(yb)
        y_pred_all.append(preds)

y_true_all = torch.cat(y_true_all).numpy()
y_pred_all = torch.cat(y_pred_all).numpy()
print("\nConfusion Matrix:\n", confusion_matrix(y_true_all, y_pred_all))
print("\nClassification Report:\n", classification_report(y_true_all, y_pred_all, digits=4))

# ====== 9) שמירת scaler לפרודקשן ======
# שמירה פשוטה של הפרמטרים של ה-StandardScaler
scaler_artifact = {
    "mean_": scaler.mean_.tolist(),
    "scale_": scaler.scale_.tolist(),
}
with open("scaler.json", "w", encoding="utf-8") as f:
    json.dump(scaler_artifact, f, ensure_ascii=False, indent=2)

print("\nSaved: best_mlp.pth and scaler.json")

# ====== 10) פונקציית חיזוי לשימוש בהמשך ======
def load_scaler(path="scaler.json"):
    with open(path, "r", encoding="utf-8") as f:
        obj = json.load(f)
    sc = StandardScaler()
    sc.mean_ = np.array(obj["mean_"], dtype=np.float64)
    sc.scale_ = np.array(obj["scale_"], dtype=np.float64)
    sc.n_features_in_ = sc.mean_.shape[0]
    return sc

def predict_embeddings(emb_list, model_path="best_mlp.pth", scaler_path="scaler.json"):
    """
    emb_list: list of embedding vectors (each is list/np.array of length d)
    returns: np.array of predicted class ids
    """
    sc = load_scaler(scaler_path)
    Xn = sc.transform(np.array(emb_list, dtype=np.float32))
    mdl = MLP(Xn.shape[1], num_classes)
    mdl.load_state_dict(torch.load(model_path, map_location="cpu"))
    mdl.eval()
    with torch.no_grad():
        logits = mdl(torch.from_numpy(Xn).float())
        preds = torch.argmax(logits, dim=1).numpy()
    return preds


Shape X: (8164, 1024) classes: [0 1 2]
Epoch 001 | loss=1.1336933374404907 | 
Epoch 002 | loss=0.9895453453063965 | 
Epoch 003 | loss=1.0620204210281372 | 
Epoch 004 | loss=1.0693849325180054 | 
Epoch 005 | loss=0.986126720905304 | 
Epoch 006 | loss=1.0886253118515015 | 
Epoch 007 | loss=0.9567194581031799 | 
Epoch 008 | loss=1.1339327096939087 | 
Epoch 009 | loss=0.8809139132499695 | 
Epoch 010 | loss=0.7725961804389954 | 
Epoch 011 | loss=0.9218748807907104 | 
Epoch 012 | loss=0.971961498260498 | 
Epoch 013 | loss=0.8989092111587524 | 
Epoch 014 | loss=0.662523627281189 | 
Epoch 015 | loss=0.6461701989173889 | 
Epoch 016 | loss=0.6011753678321838 | 
Epoch 017 | loss=0.8593766689300537 | 
Epoch 018 | loss=0.909336507320404 | 
Epoch 019 | loss=0.6443436741828918 | 
Epoch 020 | loss=0.663413941860199 | 

TEST | loss=1.0855 | acc=0.5200 | f1_macro=0.3702

Confusion Matrix:
 [[ 45  43  87]
 [ 43  46  83]
 [176 156 546]]

Classification Report:
               precision    recall  f1-score 

In [46]:
y_pred = predict_embeddings(X_test, model_path="best_mlp.pth", scaler_path="scaler.json")

In [47]:
len(y_test)

1225

In [48]:

print('סה"כ טסט:', len(y_test))
# ספירת שוויון כולל
total_equal = np.sum(y_pred == y_test)

# ספירה לפי ערכים
equal_0 = np.sum((y_pred == 0) & (y_test == 0))
equal_1 = np.sum((y_pred == 1) & (y_test == 1))
equal_2 = np.sum((y_pred == 2) & (y_test == 2))
pct = (total_equal / len(y_pred ) ) * 100

print("סה״כ כניסות שוות:", total_equal)
print("אחוז הצלחה כללי:" , pct)
print("שניהם 0:", equal_0)
print("שניהם 1:", equal_1)
print("שניהם 2:", equal_2)


סה"כ טסט: 1225
סה״כ כניסות שוות: 528
אחוז הצלחה כללי: 43.10204081632653
שניהם 0: 63
שניהם 1: 48
שניהם 2: 417


### CROSS VALIDATION

In [ ]:
# pip install torch scikit-learn pandas numpy
import ast
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# ===== קלט =====
# df = pd.read_csv("your_embeddings.csv")  # אם צריך לקרוא מקובץ
EMB_COL = "embedding"
LABEL_COL = "movement_label"

len_x = len(data.columns) - 1
X = np.vstack(data.values[:,0:len_x]).astype(np.float32)
y = data[LABEL_COL].astype(float).values
# אופציונלי: החלפת NaN ב-2
# y = np.where(np.isnan(y), 2, y)
y = y.astype(int)

num_classes = len(np.unique(y))
input_dim = X.shape[1]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
rng = np.random.default_rng(42)
torch.manual_seed(42)
if device.type == "cuda":
    torch.cuda.manual_seed_all(42)

# ===== DataSet קטן =====
class NPDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X).float()
        self.y = torch.from_numpy(y).long()
    def __len__(self): return len(self.y)
    def __getitem__(self, i): return self.X[i], self.y[i]

# ===== מודל =====
class MLP(nn.Module):
    def __init__(self, in_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 256), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(256, 128),   nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(128, num_classes)
        )
    def forward(self, x): return self.net(x)

def train_one_fold(X_tr, y_tr, X_va, y_va, max_epochs=100, lr=1e-6, wd=1e-4, patience=20):
    # איזון מחלקות לפי ה-train של הפולד
    class_counts = np.bincount(y_tr, minlength=num_classes)
    class_weights = (len(y_tr) / (num_classes * np.clip(class_counts, 1, None))).astype(np.float32)

    # DataLoaders
    train_loader = DataLoader(NPDataset(X_tr, y_tr), batch_size=64, shuffle=True)
    val_loader   = DataLoader(NPDataset(X_va, y_va), batch_size=256, shuffle=False)

    model = MLP(input_dim, num_classes).to(device)
    criterion = nn.CrossEntropyLoss(weight=torch.tensor(class_weights, device=device))
    opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)

    best_val = float("inf")
    patience_count = 0
    best_state = None

    for ep in range(1, max_epochs+1):
        # ---- train
        model.train()
        running = 0.0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            opt.zero_grad()
            logits = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=2.0)
            opt.step()
            running += loss.item() * yb.size(0)

        # ---- val
        model.eval()
        val_loss, y_true, y_pred = 0.0, [], []
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                logits = model(xb)
                loss = criterion(logits, yb)
                val_loss += loss.item() * yb.size(0)
                y_true.append(yb.cpu().numpy())
                y_pred.append(torch.argmax(logits, dim=1).cpu().numpy())
        val_loss /= len(val_loader.dataset)
        y_true = np.concatenate(y_true)
        y_pred = np.concatenate(y_pred)
        acc = accuracy_score(y_true, y_pred)
        f1m = f1_score(y_true, y_pred, average="macro")

        # Early stopping
        if val_loss < best_val - 1e-4:
            best_val = val_loss
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            patience_count = 0
        else:
            patience_count += 1
            if patience_count >= patience:
                break

    # שחזור best
    model.load_state_dict(best_state)
    return model

# ===== Cross Validation =====
k = 1  # אפשר להגדיל/להקטין
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

accs, f1s = [], []

for fold, (idx_tr, idx_va) in enumerate(skf.split(X, y), 1):
    X_tr, y_tr = X[idx_tr], y[idx_tr]
    X_va, y_va = X[idx_va], y[idx_va]

    # סטנדרטיזציה בכל פולד (פיט רק על train!)
    scaler = StandardScaler(with_mean=True, with_std=True)
    X_tr_n = scaler.fit_transform(X_tr)
    X_va_n = scaler.transform(X_va)

    model = train_one_fold(X_tr_n, y_tr, X_va_n, y_va)

    # הערכה סופית בפולד
    model.eval()
    with torch.no_grad():
        logits = model(torch.from_numpy(X_va_n).float().to(device)).cpu().numpy()
    y_pred = logits.argmax(axis=1)
    acc = accuracy_score(y_va, y_pred)
    f1m = f1_score(y_va, y_pred, average="macro")
    accs.append(acc); f1s.append(f1m)
    print(f"Fold {fold}/{k} -> ACC={acc:.4f} | F1_macro={f1m:.4f}")

print("\n=== CV summary ===")
print(f"ACC: mean={np.mean(accs):.4f}  std={np.std(accs):.4f}")
print(f"F1m: mean={np.mean(f1s):.4f}  std={np.std(f1s):.4f}")



Fold 1/5 -> ACC=0.5168 | F1_macro=0.3649
Fold 2/5 -> ACC=0.4985 | F1_macro=0.3574
Fold 3/5 -> ACC=0.5070 | F1_macro=0.3782
Fold 4/5 -> ACC=0.5321 | F1_macro=0.3795


In [23]:
with torch.no_grad():
    logits = model(torch.from_numpy(X_test).float().to(device)).cpu().numpy()
y_pred = logits.argmax(axis=1)

In [24]:
# ספירת שוויון כולל
total_equal = np.sum(y_pred == y_test)

# ספירה לפי ערכים
equal_0 = np.sum((y_pred == 0) & (y_test == 0))
equal_1 = np.sum((y_pred == 1) & (y_test == 1))
equal_2 = np.sum((y_pred == 2) & (y_test == 2))
pct = (total_equal / len(y_pred ) ) * 100

print('סה"כ כניסות:',len(y_pred ))
print("סה״כ כניסות שוות:", total_equal)
print("אחוז הצלחה כללי:" , pct)
print("שניהם 0:", equal_0)
print("שניהם 1:", equal_1)
print("שניהם 2:", equal_2)


סה"כ כניסות: 1232
סה״כ כניסות שוות: 701
אחוז הצלחה כללי: 56.899350649350644
שניהם 0: 115
שניהם 1: 63
שניהם 2: 523
